In [ ]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import plot_model
from keras.models import Model
from keras import backend as K


from keras.utils import multi_gpu_model
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import tensorflow as tf

import gc

import time


%matplotlib inline

In [ ]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True

hdf5_file = h5py.File(hdf5_path, "r")

if subtract_mean:
    mm = hdf5_file["train_mean"][...,0]
    mm = mm[np.newaxis, ...]

data_num = hdf5_file["train_flow"].shape[0]
    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

x_train = hdf5_file["train_flow"][...,0]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:,...]
hdf5_file.close()
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")


x_test = hdf5_file["IoT_flow"][...,0]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:,...]

hdf5_file.close()

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

#define number of packets used to train for each device
packets = 200
x_train = x_train[:,:packets,:]
x_test = x_test[:,:packets,:]

input_shape = (x_train.shape[1], x_train.shape[2])
    
    
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [ ]:
AccScores = []
Times = []
LossScores = []
#AucScores = []
#F1Scores = []
#PrecisionScores = []
t2 = time.time()
for i in range (1,50):
    num_classes = 2
    high = 80
    
    filter_lenghts =  [int(i) for i in np.arange(2,high,2)]
    #print(filter_lenghts)
    convs = []
    maxlen = packets
    batch_size = 1024
    epochs = 30
    numFilters=128
    activations= "sigmoid"
    dropoutVal = 0.2485206320388983
    lr = 0.001362881122433337
    adam = keras.optimizers.Adam(lr=lr)


    optim = adam
    
    input_flow = Input(shape=input_shape)

    convs= {}
    mpoolings = {}
    flattens = {}
    convs_out = []
    for i in filter_lenghts:
        convs[str(i)+'_convolution']=Conv1D(filters=numFilters,kernel_size=i,padding="valid",activation=activations,strides=1)(input_flow)

        mpoolings[str(i)+'_maxpooling'] = MaxPooling1D(pool_size= maxlen - i + 1)(convs[str(i)+'_convolution'])
        flattens[str(i)+'_flattenout'] = Flatten()(mpoolings[str(i)+'_maxpooling'])
        convs_out.append(flattens[str(i)+'_flattenout'])
    out = Concatenate()(convs_out)
    dropout = Dropout(dropoutVal)(out)
    dense = Dense(64, activation='relu')(dropout)
    dense2 = Dense(32, activation='relu')(dense)
    dropout2 = Dropout(dropoutVal)(dense2)
    end = Dense(num_classes, activation='softmax')(dropout2)

    model = Model(inputs=input_flow, outputs=end) 
    #model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
    
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    t0 = time.time()
    model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    t1 = time.time()
    total_run = t1-t0
    Times.append(total_run)
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.5f%%" % (model.metrics_names[0], scores[0]))
    print('Fit and Evaluate Time:',total_run)
    AccScores.append(scores[1] * 100)
    LossScores.append(scores[0])
    del model
    gc.collect()
    K.clear_session()
    gc.collect()

    
print("%.5f%% (+/- %.5f%%)" % (np.mean(AccScores), np.std(AccScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(LossScores), np.std(LossScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(Times), np.std(Times)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(AucScores), np.std(AucScores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(F1Scores), np.std(F1Scores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(PrecisionScores), np.std(PrecisionScores)))
t3 = time.time()
total_time = t3-t2
print('Time to do everything:',total_time)


In [ ]:
AccScores

In [ ]:
LossScores

In [ ]:
Times